In [1]:
from torchvision.models import *
import wandb
from sklearn.model_selection import train_test_split
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import *
import torch,torchvision
from tqdm import tqdm
device = 'cuda'
PROJECT_NAME = 'Vechicle-Dataset'

In [2]:
def load_data():
    labels = {}
    labels_r = {}
    data = []
    idx = -1
    for folder in tqdm(os.listdir('./data/')):
        idx += 1
        labels[folder] = idx
        labels_r[idx] = folder
        for file in os.listdir(f'./data/{folder}/'):
            img = cv2.imread(f'./data/{folder}/{file}')
            img = cv2.resize(img,(112,112))
            img = img / 255.0
            data.append([img,idx])
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(d[1])
    X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle=False,test_size=0.0625)
    return X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data

In [3]:
X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data = load_data()

100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 48.08it/s]


In [4]:
X_train = torch.from_numpy(np.array(X_train)).view(-1,3,112,112).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).view(-1,3,112,112).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

In [5]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y.long())
    return loss.item()

In [6]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.argmax(pred))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [17]:
model = googlenet(pretrained=True).to(device)

In [27]:
model.fc = Linear(1024,len(labels)-1)

In [28]:
idx

480

In [29]:
len(labels)

10

In [30]:
labels

{'train': 0,
 'bus': 1,
 'boat': 2,
 'cycle': 3,
 'truck': 4,
 'scooty': 5,
 'car': 6,
 'helicopter': 7,
 'plane': 8,
 'bike': 9}

In [31]:
criterion = CrossEntropyLoss()

In [32]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [33]:
epochs = 100

In [34]:
batch_size = 32

In [26]:
wandb.init(project=PROJECT_NAME,name='baseline-1')
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].to(device)
        y_batch = y_train[idx:idx+batch_size].to(device)
        model.to(device)
        preds = model(X_batch.to(device))
        loss = criterion(preds,y_batch.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+get_loss(model,X_batch,y_batch,criterion))/2})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train)+get_accuracy(model,X_batch,y_batch))/2})
    wandb.log({'Val ACC':get_accuracy(model,X_test,y_test)})
wandb.finish()

100%|█████████████████████████████████████████| 100/100 [01:42<00:00,  1.02s/it]


Loss,0.85295
_runtime,108
_timestamp,1631363114
_step,399
Val Loss,8.8575
Acc,72.05
Val ACC,0.0


Loss,█▇▆▅▅▅▃▃▁▁▁▁▁▅▄▂▂▂▁▁▁▂▂▂▂▂▂▂▂▂▂▂▄▃▂▂▁▁▁▂
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Val Loss,▁▁▂▄▄▃▄▄▅▅▆▆▆▆▃▆▆▇▇▇███████████▅▃▆▆▇▇▇▇▇
Acc,▁▁▃▃▃▆▇▇█████▅▆▇▇▇▇▇▇▇▇▆▆▆▆▆▆▇▆▆▅▅▆▆▇▇▇▇
Val ACC,▁▁▁▁▁▁▁███▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
